In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from util_io import (
    init, finalize, dump_conf, assigned_day_to_family_on_day, assigned_day_to_occupancy
)
from util_cost import (
    cal_total, n_people, family_id_choice_to_pref_cost, cal_total_preference, cal_total_accounting,
    nd_ndp1_to_account_penality, fixed_family_cost, fixed_person_cost
)
from util_cost import choices as family_pref
from util_check import deep_check, check_valid_all

In [2]:
# constants #
N_families = 5000
N_days = 100
N_min_people = 125
N_max_people = 300
# constants #

# params #
#path_init_conf =     '../output/m08-improved-test.csv'
#path_init_conf =     '../input/another_pytorch_implementation.csv'
#path_dump_improved = '../output/m14-improved-v2.csv' # output solution

num_cpu_cores = 6
#time_limit = -1 # unlimited
time_limit = 3*60*60*1000  # in ms

# occupancy_diff = 2  # +- the occupancy of input solution for each day
max_family_rank = 3  # maximum number of rank of the preference days for each family
# use_hint = True      # use current input as hint
# occupancy_count_as_variables = False  # use occupancy_counts as variable
# min_choice_0_families = 3000   # minimum number of families that are at their choice 0
target_pref_cost = 62868

In [3]:
max_family_size = int(max(n_people))
min_family_size = int(min(n_people))
max_family_size, min_family_size

(8, 2)

In [4]:
possible_family_size = range(min_family_size, max_family_size+1)

In [5]:
possible_family_rank = range(max_family_rank)

In [6]:
n_family_size = {s: int((n_people==s).sum()) for s in possible_family_size}

In [7]:
n_family_size

{2: 717, 3: 981, 4: 1451, 5: 899, 6: 494, 7: 301, 8: 157}

In [8]:
total_people = int(n_people.sum())
total_people

21003

## Ortools - CBC MIP solver

In [9]:
from ortools.linear_solver import pywraplp

In [10]:
solver = pywraplp.Solver('', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

In [11]:
if num_cpu_cores > 0:
    print('Set num threads:', solver.SetNumThreads(num_cpu_cores))
if time_limit > 0:
    print('Set time limit:', solver.SetTimeLimit(time_limit))

Set num threads: True
Set time limit: None


In [12]:
n_familis_size_at_rank = {
    s: [solver.IntVar(0, n_family_size[s], 'f_%i_%i' % (s, rank))
        for rank in possible_family_rank]
    for s in possible_family_size 
}

In [13]:
sum_family_cost = solver.Sum([
    solver.Sum([
        n * int(fixed_family_cost[rank]) for rank, n in enumerate(n_familis_at_rank)
    ]) for n_familis_at_rank in n_familis_size_at_rank.values()
])

In [14]:
#sum_family_cost.GetCoeffs()

In [15]:
sum_people_cost = solver.Sum([
    solver.Sum([
        n * int(fixed_person_cost[rank] * size) for rank, n in enumerate(n_familis_at_rank)
    ]) for size, n_familis_at_rank in n_familis_size_at_rank.items()
])

In [16]:
#sum_people_cost.GetCoeffs()

In [17]:
solver.Add(sum_family_cost + sum_people_cost == target_pref_cost);

In [18]:
for size, n_families in n_family_size.items():
    solver.Add(solver.Sum(n_familis_size_at_rank[size]) == n_families)

In [19]:
# for rank in range(max_family_rank):
#     solver.Add(n_familis_at_rank[rank] * max_family_size >= n_people_at_rank[rank])
#     solver.Add(n_familis_at_rank[rank] * min_family_size <= n_people_at_rank[rank])

In [20]:
solver.Minimize(sum_family_cost + sum_people_cost)

In [21]:
solver.NumVariables()

21

In [22]:
%%time
# Solve
sol = solver.Solve()

resdict = {0:'OPTIMAL', 1:'FEASIBLE', 2:'INFEASIBLE', 3:'UNBOUNDED', 
           4:'ABNORMAL', 5:'MODEL_INVALID', 6:'NOT_SOLVED'}
print('Result: ', resdict[sol])
print('Total cost = ', solver.Objective().Value())
print("Time = ", solver.WallTime(), " milliseconds")

Result:  OPTIMAL
Total cost =  62868.0
Time =  96  milliseconds
CPU times: user 24.6 ms, sys: 3.93 ms, total: 28.6 ms
Wall time: 28.2 ms


In [23]:
solver.VerifySolution(0.0001, False)

True

In [24]:
n_familis_size_at_rank_sol = {
    s: [n_familis_size_at_rank[s][rank].solution_value()
        for rank in possible_family_rank]
    for s in possible_family_size 
}

In [25]:
n_familis_size_at_rank_sol

{2: [717.0, 0.0, 0.0],
 3: [981.0, 0.0, 0.0],
 4: [1451.0, 0.0, 0.0],
 5: [899.0, 0.0, 0.0],
 6: [78.0, 0.0, 416.0],
 7: [292.0, 9.0, 0.0],
 8: [0.0, 0.0, 157.0]}

In [26]:
family_cost_sol = sum([
    n_familis_size_at_rank_sol[size][rank] * fixed_family_cost[rank]
    for size in possible_family_size for rank in possible_family_rank
])
people_cost_sol = sum([
    n_familis_size_at_rank_sol[size][rank] * fixed_person_cost[rank] * size
    for size in possible_family_size for rank in possible_family_rank
])
print(family_cost_sol, people_cost_sol, family_cost_sol+people_cost_sol, target_pref_cost)

29100.0 33768.0 62868.0 62868
